In [1]:
# Import our dependencies
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf
import random
from datetime import date

In [2]:
pip install yahoo_fin


Note: you may need to restart the kernel to use updated packages.


In [3]:
from yahoo_fin.stock_info import get_data

In [4]:

# ***************************
# ***                     ***
# ***    SET VARIABLES    ***
# ***                     ***
# ***************************

days_in_future_that_result_is_calculated = 252
print(f"Days in future that result is calculated: {days_in_future_that_result_is_calculated}")

reversion_low_value = 12
print(f"reversion_low_value: {reversion_low_value}")

reversion_high_value = 20
print(f"reversion_high_value: {reversion_high_value}")

optimization_max_epochs = 5
print(f"optimization_max_epochs: {optimization_max_epochs}")

# Max 10:
optimization_max_hidden_layers = 5
print(f"optimization_max_hidden_layers: {optimization_max_hidden_layers}")

optimization_max_nodes = 5


activation_type = 'relu'

# Max 10:
num_layers = 2

num_epochs = 5

num_nodes = 5


# SCALE THE DATA? "Y" or "N":
scale_data = "Y"


# TICKER SYMBOLS TO PROCESS:

# 21 symbols to pair up, plus 2 result tickers to test output,
# so 210 unique pairs * 2 outputs = 420 combinations are 
# looped through and be tested

symbol_eval_list = [
                        '^GSPC',
                        '^DJI',
                        '^IXIC',
                        '^RUT',
                        'XLC',
                        'XLY',
                        'XLP',
                        'XLE',
                        'XLF',
                        'XLV',
                        'XLI',
                        'XLB',
                        'XLRE',
                        'XLK',
                        'XLU',
                        '^VIX',
                        '^VIX3M',
                        'CL=F',
                        '^TNX',
                        'BTC-USD',
                        '^CMC200',
                        ]

symbol_result_list = ['SPY'
                    , 'USO'
                     ]

input_symbol_01 = ""
input_symbol_02 = ""
input_symbol_03 = ""
output_count = 0

# Triple-loop through all of the symbol_eval_list and all of the symbol_result_list
# and determine all possible combinations. Write all combos to a list. 
# Then auto-run machine learning neural networks for each
# one of them by looping through the list that was created.
# Use the results to determine best candidates for further analysis.
# The result is 420 unique combinations to test.

# Outer loop through the smaller group of ticker symbols symbol_result_list
for x3 in symbol_result_list:
  input_symbol_03 = x3
  pairings = []
  for x1 in symbol_eval_list:
    # Then get a ticker symbol from symbol_eval_list
    input_symbol_01 = x1
    for x2 in symbol_eval_list:
      # get another symbol to pair up from ymbol_eval_list 
      input_symbol_02 = x2
      # if the two symbols are not the same, proceed
      if input_symbol_01 != input_symbol_02:
        pairing_found = 'N'
        # loop through the found pairings and check if that pairing already exists (also check the reverse order)
        for i in range(len(pairings)):
          if (pairing_found == 'N'):
            if (pairings[i] == [x1, x2]) | (pairings[i] == [x2, x1]):
              pairing_found = 'Y'
        # if no pairing has already been writted out, we can process it, so write it now so it won't be processed again and then do all the procesing
        if (pairing_found == 'N'):
          pairings.append([x1, x2])
          print(f"BEGIN PROCESS")

          # Load ticker1 historical data from Yahoo Finance API
          today = date.today()
          curr_date = today.strftime("%m/%d/%Y")
          df_01 = get_data(input_symbol_01, start_date="01/01/2000", end_date=curr_date, index_as_date = False, interval="1d")
          df_01_min_date = (df_01.iloc[0]["date"])

          # Load ticker2 historical data from Yahoo Finance API
          from datetime import date
          today = date.today()
          curr_date = today.strftime("%m/%d/%Y")
          df_02 = get_data(input_symbol_02, start_date="01/01/2000", end_date=curr_date, index_as_date = False, interval="1d")
          df_02_min_date = (df_02.iloc[0]["date"])

          # Load ticker3 historical data from Yahoo Finance API
          from datetime import date
          today = date.today()
          curr_date = today.strftime("%m/%d/%Y")
          df_03 = get_data(input_symbol_03, start_date="01/01/2000", end_date=curr_date, index_as_date = False, interval="1d")
          df_03_min_date = (df_03.iloc[0]["date"])

          # Find the first date in each dataframe and get the latest date, that way all dataframes start on the same date when concatinated
          latest_date = df_01_min_date
          if df_02_min_date > latest_date:
            latest_date = df_02_min_date
          if df_03_min_date > latest_date:
            latest_date = df_03_min_date
          print(f"latest_date: {latest_date}")
          df_01 = df_01[df_01.date >= latest_date]
          df_02 = df_02[df_02.date >= latest_date]
          df_03 = df_03[df_03.date >= latest_date]
          
          output_count+=1
          print(f"OUTPUT COUNT: {output_count}")
          # set the column names for this run, based on the ticker symbols
          col01 = (f"{input_symbol_01}_close")
          col02 = (f"{input_symbol_02}_close")
          col03 = (f"{input_symbol_03}_close")

          # clean and merge the 3 dataframes, and add some new cols
          df = df_01[['date', 'close']].copy()
          df = df.rename(columns={'close': col01})
          df = df.merge(df_02, on='date')
          df = df.rename(columns={'close': col02})
          df = df.drop(columns=['open', 'high', 'low', 'adjclose', 'volume', 'ticker'])
          df['percentage'] = ""
          df['percentage_boolean'] = ""
          df['reversion_boolean'] = ""
          df = df.merge(df_03, on='date')
          df = df.rename(columns={'close': col03})
          df = df.drop(columns=['open', 'high', 'low', 'adjclose', 'volume', 'ticker'])
          df['future_result'] = ""
          df['future_result_boolean'] = ""

          # loop through the df to populate the new columns
          for i in range(len(df)):
            if (i < (len(df))):
              df.percentage[i] = (df[col02][i]/df[col01][i])
              if ( (df[col01][i] > reversion_low_value) & (df[col01][i] < reversion_high_value) ):
                df.reversion_boolean[i] = 1
              else:
                df.reversion_boolean[i] = 0
            if (i < (len(df)-days_in_future_that_result_is_calculated)):
              df.future_result[i] = ((df[col03][i+days_in_future_that_result_is_calculated]-df[col03][i])/df[col03][i])

          df = df[df['future_result'] != ""]


          # For the model, pick which cols will be fed into 'X' as inputs
          input_column_list_for_X = [
              input_symbol_01+"_close"
            , 
            input_symbol_02+"_close"
            # , 
            # "percentage"
            # ,
            # "percentage_boolean"
            # ,
            # "reversion_boolean"
            ]
          print (f"input_column_list_for_X: {input_column_list_for_X}")

          # For the model, pick which col will be the 'y' result
          output_col_for_y = [
              # "future_result"
              "future_result_boolean"
              ]
          print (f"output_col_for_y: {output_col_for_y}")

          # make sure numeric
          df["percentage"] = pd.to_numeric(df["percentage"])
          df["reversion_boolean"] = pd.to_numeric(df["reversion_boolean"])
          df["future_result"] = pd.to_numeric(df["future_result"])

          # set more new cols
          for i in range(len(df)):
              if (df['percentage'][i] > 1):
                df.percentage_boolean[i] = 1
              else:
                df.percentage_boolean[i] = 0
              if (df['future_result'][i] > 0):
                df.future_result_boolean[i] = 1
              else:
                df.future_result_boolean[i] = 0

          df["future_result_boolean"] = pd.to_numeric(df["future_result_boolean"])
          df["percentage_boolean"] = pd.to_numeric(df["percentage_boolean"])

          # Remove target 'y' from features data
          y = df[output_col_for_y].values

          # Pick columns to use for input 'X'
          X = df[input_column_list_for_X].values

          # Split the preprocessed data into a training and testing dataset
          X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

          # Create a StandardScaler instances
          scaler = StandardScaler()

          # Fit the StandardScaler
          X_scaler = scaler.fit(X_train)

          # Scale the data
          if scale_data == "N":
            import numpy as np
            X_train = np.asarray(X_train).astype(np.float64)
            X_train_scaled = X_train
            X_test = np.asarray(X_test).astype(np.float64)
            X_test_scaled = X_test
          else:
            X_train_scaled = X_scaler.transform(X_train)
            X_test_scaled = X_scaler.transform(X_test)

          # Compile, Train and Evaluate the Model

          number_input_features = len(X_train[0])

          nn = tf.keras.models.Sequential()

          # First hidden layer
          hidden_nodes_layer1 = num_nodes
          nn.add(
              tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation=activation_type)
          )
          if num_layers >= 2:
            # second hidden layer
            hidden_nodes_layer2 = num_nodes
            nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation=activation_type))
          if num_layers >= 3:
            # hidden layer
            hidden_nodes_layer3 = num_nodes
            nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation=activation_type))
          if num_layers >= 4:
            # hidden layer
            hidden_nodes_layer4 = num_nodes
            nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation=activation_type))
          if num_layers >= 5:
            # hidden layer
            hidden_nodes_layer5 = num_nodes
            nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation=activation_type))
          if num_layers >= 6:
            # hidden layer
            hidden_nodes_layer6 = num_nodes
            nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer6, activation=activation_type))
          if num_layers >= 7:
            # hidden layer
            hidden_nodes_layer7 = num_nodes
            nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer7, activation=activation_type))
          if num_layers >= 8:
            # hidden layer
            hidden_nodes_layer8 = num_nodes
            nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer8, activation=activation_type))
          if num_layers >= 9:
            # hidden layer
            hidden_nodes_layer9 = num_nodes
            nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer9, activation=activation_type))
          if num_layers >= 10:
            # hidden layer
            hidden_nodes_layer10 = num_nodes
            nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer10, activation=activation_type))

          # Output layer
          nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

          # Check the structure of the model
          nn.summary()

          # Compile the model
          nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
          # Train the model
          fit_model = nn.fit(X_train_scaled,y_train,epochs=num_epochs)

          # Evaluate the model using the test data
          model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)

          # Need to document well the RESULTS from EACH run - this output will be just one of hundreds of loops in the output
          print(f"RESULTS")
          print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
          print(f"Symbol 01: {input_symbol_01}")
          print(f"Symbol 02: {input_symbol_02}")
          print(f"Symbol 03: {input_symbol_03}")
          print(f"Scale Data = {scale_data}")
          print(f"activation_type = {activation_type}")
          print(f"num_epochs = {num_epochs}")
          print(f"num_layers = {num_layers}")
          print(f"Layer 1 Nodes = {hidden_nodes_layer1}")
          if num_layers >= 2:
            print(f"Layer 2 Nodes = {hidden_nodes_layer2}")
          if num_layers >= 3:
            print(f"Layer 3 Nodes: {hidden_nodes_layer3}")
          if num_layers >= 4:
            print(f"Layer 4 Nodes = {hidden_nodes_layer4}")
          if num_layers >= 5:
            print(f"Layer 5 Nodes = {hidden_nodes_layer5}")
          if num_layers >= 6:
            print(f"Layer 6 Nodes = {hidden_nodes_layer6}")
          if num_layers >= 7:
            print(f"Layer 7 Nodes = {hidden_nodes_layer7}")
          if num_layers >= 8:
            print(f"Layer 8 Nodes = {hidden_nodes_layer8}")
          if num_layers >= 9:
            print(f"Layer 9 Nodes = {hidden_nodes_layer9}")
          if num_layers >= 10:
            print(f"Layer 10 Nodes = {hidden_nodes_layer10}")
          
          print("")
          print("")
          print("")

Days in future that result is calculated: 252
reversion_low_value: 12
reversion_high_value: 20
optimization_max_epochs: 5
optimization_max_hidden_layers: 5
BEGIN PROCESS
latest_date: 2000-01-03 00:00:00
OUTPUT COUNT: 1


C:\Users\mlech\AppData\Local\Temp\ipykernel_24428\3677634320.py:165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.percentage[i] = (df[col02][i]/df[col01][i])
C:\Users\mlech\AppData\Local\Temp\ipykernel_24428\3677634320.py:169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.reversion_boolean[i] = 0
C:\Users\mlech\AppData\Local\Temp\ipykernel_24428\3677634320.py:171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.future_result[i]

input_column_list_for_X: ['^GSPC_close', '^DJI_close']
output_col_for_y: ['future_result_boolean']


C:\Users\mlech\AppData\Local\Temp\ipykernel_24428\3677634320.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.percentage_boolean[i] = 1
C:\Users\mlech\AppData\Local\Temp\ipykernel_24428\3677634320.py:211: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.future_result_boolean[i] = 0
C:\Users\mlech\AppData\Local\Temp\ipykernel_24428\3677634320.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.future_result_boolean[i] = 1


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5)                 15        
                                                                 
 dense_1 (Dense)             (None, 5)                 30        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 51 (204.00 Byte)
Trainable params: 51 (204.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
141/141 [==============================] - 1s 909us/step - loss: 0.6276 - accuracy: 0.7375
Epoch 2/5
141/141 [==============================] - 0s 896us/step - loss: 0.5853 - accuracy: 0.7348
Epoch 3/5
141/141 [==============================] - 0s 881us/step - loss: 0.5693 

...

Model: "sequential_418"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1254 (Dense)          (None, 5)                 15        
                                                                 
 dense_1255 (Dense)          (None, 5)                 30        
                                                                 
 dense_1256 (Dense)          (None, 1)                 6         
                                                                 
Total params: 51 (204.00 Byte)
Trainable params: 51 (204.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
28/28 [==============================] - 0s 989us/step - loss: nan - accuracy: 0.3821
Epoch 2/5
28/28 [==============================] - 0s 952us/step - loss: nan - accuracy: 0.3788
Epoch 3/5
28/28 [==============================] - 0s 961us/step - loss: nan - accuracy:

C:\Users\mlech\AppData\Local\Temp\ipykernel_24428\3677634320.py:165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.percentage[i] = (df[col02][i]/df[col01][i])
C:\Users\mlech\AppData\Local\Temp\ipykernel_24428\3677634320.py:169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.reversion_boolean[i] = 0
C:\Users\mlech\AppData\Local\Temp\ipykernel_24428\3677634320.py:171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.future_result[i]

input_column_list_for_X: ['BTC-USD_close', '^CMC200_close']
output_col_for_y: ['future_result_boolean']


C:\Users\mlech\AppData\Local\Temp\ipykernel_24428\3677634320.py:207: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.percentage_boolean[i] = 0
C:\Users\mlech\AppData\Local\Temp\ipykernel_24428\3677634320.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.future_result_boolean[i] = 1
C:\Users\mlech\AppData\Local\Temp\ipykernel_24428\3677634320.py:211: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.future_result_boolean[i] = 0


Model: "sequential_419"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1257 (Dense)          (None, 5)                 15        
                                                                 
 dense_1258 (Dense)          (None, 5)                 30        
                                                                 
 dense_1259 (Dense)          (None, 1)                 6         
                                                                 
Total params: 51 (204.00 Byte)
Trainable params: 51 (204.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
28/28 [==============================] - 0s 965us/step - loss: nan - accuracy: 0.3676
Epoch 2/5
28/28 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.3788
Epoch 3/5
28/28 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.3